In [ ]:
# Setup: Ensure the animal_shelter module is in the Python path
import sys
from pathlib import Path

candidate = Path.cwd()
project_root = None
# Search up for the project root to ensure module can be imported
for _ in range(6):
    if (candidate / "animal_shelter").is_dir():
        project_root = candidate
        break
    candidate = candidate.parent

if project_root and str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

In [ ]:
# Imports
from dash import Dash, dcc, html
from dash import dash_table
from dash.dependencies import Input, Output
import dash_leaflet as dl
import plotly.express as px
import pandas as pd

# CRUD module
from animal_shelter.animal_shelter import AnimalShelter

In [ ]:
# Model: Connect to MongoDB and fetch all records
# Per assignment, hardcode aacuser credentials here
# If .env is loaded by AnimalShelter, these will override
AAC_USER = "aacuser"
AAC_PASS = "SECRET"  # Using init-mongo default; override if your .env differs

shelter = AnimalShelter(user=AAC_USER, password=AAC_PASS)

# Retrieve all documents
records = shelter.read({})

# Debug prints to verify data
print(f"Records fetched: {len(records)}")

# Build DataFrame
df = pd.DataFrame.from_records(records)
# Drop MongoDB _id to avoid serialization issues
if "_id" in df.columns:
    df.drop(columns=["_id"], inplace=True)

# Replace missing values to avoid serialization issues in DataTable
df = df.fillna("")
print(f"Columns: {list(df.columns)[:12]} ... ({len(df.columns)} total)")
print(df.head(2))

# Fallbacks for missing lat/lon column names
# Common AAC columns: 'location_lat', 'location_long'
lat_col = None
lon_col = None
for candidate in ["location_lat", "lat", "latitude", "y"]:
    if candidate in df.columns:
        lat_col = candidate
        break
for candidate in ["location_long", "lon", "longitude", "x"]:
    if candidate in df.columns:
        lon_col = candidate
        break

# Ensure we have coordinates; if not, create NaNs to avoid crashes
if lat_col is None:
    df["location_lat"] = pd.NA
    lat_col = "location_lat"
if lon_col is None:
    df["location_long"] = pd.NA
    lon_col = "location_long"

# Choose a concise set of visible columns for readability
default_visible_columns = [
    'animal_id', 'name', 'animal_type', 'breed', 'outcome_type', 'datetime', 'sex_upon_outcome'
]
visible_columns = [c for c in default_visible_columns if c in df.columns]
# Always include coordinates for the map but hide them from the UI
table_columns = visible_columns + [col for col in [lat_col, lon_col] if col in df.columns]
hidden_columns = [col for col in [lat_col, lon_col] if col in table_columns]

# Build an initial subset for performance (optional)
# df_view = df.head(1000)

unique_id = "Dave Mobley"


In [ ]:
# View: Build the dashboard layout (Dash, matching Module Five style)
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
app = Dash(__name__, external_stylesheets=external_stylesheets, assets_folder=str(project_root / 'assets'))

app.layout = html.Div(
    className='container',
    children=[
        html.H1('SNHU CS-340 Dashboard - Module 6', style={'textAlign': 'center'}),
        html.P(unique_id, style={'textAlign': 'center'}),
        html.Hr(),
        html.Div(
            id='table-container',
            children=[
                dash_table.DataTable(
                    id='datatable-id',
                    columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in table_columns],
                    data=(df[table_columns].to_dict('records') if all(col in df.columns for col in table_columns) else df.to_dict('records')),
                    page_action='none',
                    sort_action='native',
                    filter_action='native',
                    row_selectable='single',
                    selected_rows=[0],
                    fixed_rows={'headers': True},
                    virtualization=False,
                    fill_width=False,
                    style_table={'overflowX': 'auto', 'overflowY': 'auto', 'height': '60vh', 'width': '100%'},
                    style_cell={
                        'minWidth': '100px', 'width': '140px', 'maxWidth': '280px',
                        'whiteSpace': 'nowrap',
                        'textOverflow': 'ellipsis',
                        'overflow': 'hidden',
                        'padding': '8px',
                        'fontFamily': '-apple-system, BlinkMacSystemFont, "Segoe UI", Roboto, Helvetica, Arial, sans-serif',
                        'fontSize': 13, 'lineHeight': '18px',
                        'border': '1px solid #eaeaea'
                    },
                    style_header={
                        'backgroundColor': '#2a5bd7',
                        'color': 'white',
                        'fontWeight': 'bold',
                        'border': '0px'
                    },
                    style_cell_conditional=[
                        {'if': {'column_id': 'animal_id'}, 'width': '120px'},
                        {'if': {'column_id': 'name'}, 'minWidth': '160px', 'width': '180px', 'maxWidth': '220px'},
                        {'if': {'column_id': 'breed'}, 'minWidth': '180px', 'width': '200px'},
                        {'if': {'column_id': 'animal_type'}, 'width': '120px'},
                        {'if': {'column_id': 'outcome_type'}, 'width': '140px'},
                        {'if': {'column_id': 'datetime'}, 'width': '160px'},
                        {'if': {'column_id': lat_col}, 'display': 'none'},
                        {'if': {'column_id': lon_col}, 'display': 'none'},
                    ],
                    style_as_list_view=True,
                )
            ]
        ),
        html.Br(),
        html.Hr(),
        html.Div(id='map-container', children=[html.Div(id='map-id')])
    ]
)

In [ ]:
# Controller: interactions and map update

# Highlight selected columns in table
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns'),
     Input('datatable-id', 'selected_rows')]
)
def update_styles(selected_columns, selected_rows):
    selected_columns = selected_columns or []
    selected_rows = selected_rows or []

    col_styles = [{
        'if': {'column_id': col},
        'background_color': '#E8F2FF'
    } for col in selected_columns]

    row_styles = [{
        'if': {'row_index': idx},
        'background_color': '#FFF6E5'
    } for idx in selected_rows]

    return col_styles + row_styles


# Map update uses derived data and selected rows
@app.callback(
    Output('map-id', 'children'),
    [Input('datatable-id', 'derived_virtual_data'),
     Input('datatable-id', 'derived_virtual_selected_rows')]
)
def update_map(viewData, index):
    records_local = viewData if viewData is not None else df.to_dict('records')
    dff = pd.DataFrame.from_records(records_local)

    if dff.empty:
        return html.Div("No data available for map.")

    # Determine target row index
    row = 0 if not index else index[0]
    row = max(0, min(row, len(dff) - 1))

    # Try the selected row, otherwise find first row with valid coords
    lat = dff.iloc[row].get(lat_col)
    lon = dff.iloc[row].get(lon_col)
    if pd.isna(lat) or pd.isna(lon):
        valid = dff[dff[lat_col].notna() & dff[lon_col].notna()]
        if not valid.empty:
            row = valid.index[0]
            lat = valid.iloc[0][lat_col]
            lon = valid.iloc[0][lon_col]

    # Default center to Austin, TX
    center = [30.75, -97.48]
    zoom = 10
    children = [dl.TileLayer(id="base-layer-id")]

    try:
        if pd.notna(lat) and pd.notna(lon):
            center = [float(lat), float(lon)]
            zoom = 14
            name = str(dff.iloc[row].get('name', 'Unknown'))
            animal_type = str(dff.iloc[row].get('animal_type', 'Animal'))
            breed = str(dff.iloc[row].get('breed', 'Unknown breed'))
            outcome = str(dff.iloc[row].get('outcome_type', '') or '')
            popup = dl.Popup([
                html.Div([
                    html.H4(name, style={'margin': '0 0 6px 0'}),
                    html.Div(animal_type, style={'fontSize': '14px', 'color': '#555'}),
                    html.Div(breed, style={'fontSize': '14px', 'color': '#555'}),
                    html.Div(outcome, style={'fontSize': '13px', 'color': '#777'})
                ])
            ])
            marker = dl.Marker(position=center, children=[dl.Tooltip(breed), popup])
            children.append(marker)
    except Exception:
        pass

    return [
        dl.Map(
            style={'width': '100%', 'height': '70vh'},
            center=center, zoom=zoom, children=children
        )
    ]

In [ ]:
# Run the app (use Dash in Jupyter the same way as Module Five)
app.run(jupyter_mode='external')